In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [34]:
data=pd.read_csv('/content/cars.csv')

In [35]:
data.head()

,Acceleration,Cylinders,Displacement,Horsepower,Model_Year,Weight,Origin,MPG
0,12.0,8,307.0,130.0,70,3504,USA,18.0
1,11.5,8,350.0,165.0,70,3693,USA,15.0
2,11.0,8,318.0,150.0,70,3436,USA,18.0
3,12.0,8,304.0,150.0,70,3433,USA,16.0
4,10.5,8,302.0,140.0,70,3449,USA,17.0


In [36]:
data.shape

(406, 8)

In [37]:
data.info

<bound method DataFrame.info of      Acceleration  Cylinders  Displacement  Horsepower  Model_Year  Weight  \
0            12.0          8         307.0       130.0          70    3504   
1            11.5          8         350.0       165.0          70    3693   
2            11.0          8         318.0       150.0          70    3436   
3            12.0          8         304.0       150.0          70    3433   
4            10.5          8         302.0       140.0          70    3449   
..            ...        ...           ...         ...         ...     ...   
401          15.6          4         140.0        86.0          82    2790   
402          24.6          4          97.0        52.0          82    2130   
403          11.6          4         135.0        84.0          82    2295   
404          18.6          4         120.0        79.0          82    2625   
405          19.4          4         119.0        82.0          82    2720   

      Origin   MPG  
0    USA  

In [38]:
data_types=data.dtypes
data_types

Acceleration    float64
Cylinders         int64
Displacement    float64
Horsepower      float64
Model_Year        int64
Weight            int64
Origin           object
MPG             float64
dtype: object

In [39]:
data.isna().sum()

Acceleration    0
Cylinders       0
Displacement    0
Horsepower      6
Model_Year      0
Weight          0
Origin          0
MPG             8
dtype: int64

In [40]:
mean_value = data['MPG'].mean()
data['MPG'].fillna(mean_value, inplace=True)


In [41]:
mean_value = data['Horsepower'].mean()
data['Horsepower'].fillna(mean_value, inplace=True)

In [42]:
data.isna().sum()

Acceleration    0
Cylinders       0
Displacement    0
Horsepower      0
Model_Year      0
Weight          0
Origin          0
MPG             0
dtype: int64

In [43]:
X=data.drop(['MPG'],axis=1)
y=data['MPG']

In [44]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [45]:
unique_categories = X_train['Origin'].unique()
unique_categories

array(['USA    ', 'Japan  ', 'Sweden ', 'Germany', 'Italy  ', 'France ',
       'England'], dtype=object)

In [46]:
X_train['Origin']=X_train['Origin'].map({'USA    ':0,'Sweden ':1,'Germany':2,'Italy  ':3,'France ':4,'England':5,'Japan  ':6})
X_train.head()

,Acceleration,Cylinders,Displacement,Horsepower,Model_Year,Weight,Origin
395,17.0,6,262.0,85.0,82,3015,0
3,12.0,8,304.0,150.0,70,3433,0
18,9.5,8,400.0,150.0,70,3761,0
131,11.0,8,350.0,180.0,73,3664,0
118,13.5,3,70.0,90.0,73,2124,6


In [47]:
X_test['Origin']=X_test['Origin'].map({'USA    ':0,'Sweden ':1,'Germany':2,'Italy  ':3,'France ':4,'England':5,'Japan  ':6})
X_test.head()

,Acceleration,Cylinders,Displacement,Horsepower,Model_Year,Weight,Origin
70,12.0,8,400.0,175.0,72,4385,0
218,16.7,6,168.0,120.0,76,3820,2
258,12.8,8,302.0,139.0,78,3570,0
33,13.5,8,318.0,210.0,70,4382,0
42,15.5,6,250.0,100.0,71,3329,0


In [48]:
model_1 = DecisionTreeRegressor(max_depth=3)  # You can set max_depth to control the depth of the tree
model_1.fit(X_train, y_train)

model_1_preds = model_1.predict(X_test)

mse_1= mean_squared_error(y_test, model_1_preds)
print("Mean Squared Error model_1: ", mse_1)


Mean Squared Error model_1:  17.62155608498781


In [49]:
regressordt = DecisionTreeRegressor()

scores = cross_val_score(regressordt, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

for fold, score in enumerate(scores):
    print(f"Fold {fold+1}: Score = {score}")

average_score = -scores.mean()
average_score

Fold 1: Score = -14.71861014776239
Fold 2: Score = -12.028401586595521
Fold 3: Score = -10.674923076923077
Fold 4: Score = -17.159755452950883
Fold 5: Score = -10.446875000000002


13.005713052846374

In [50]:
model_2 = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust n_estimators and other hyperparameters
model_2.fit(X_train, y_train)

model_2_preds = model_2.predict(X_test)

mse_2 = mean_squared_error(y_test, model_2_preds)
print(f"Mean Squared Error model_2: {mse_2}")


Mean Squared Error model_2: 9.158102439583308


In [51]:
regressorrf = RandomForestRegressor(n_estimators=100)  # You can adjust hyperparameters here

scores = cross_val_score(regressorrf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

for fold, score in enumerate(scores):
    print(f"Fold {fold+1}: Score = {score}")

average_score = -scores.mean()
average_score

Fold 1: Score = -14.275136110264503
Fold 2: Score = -6.471452044434392
Fold 3: Score = -6.205232380986491
Fold 4: Score = -9.559178331305935
Fold 5: Score = -6.38627046572608


8.57945386654348

In [52]:

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

num_round = 100
model_3 = xgb.train(params, dtrain, num_round)

model_3_preds = model_3.predict(dtest)

mse_3 = mean_squared_error(y_test, model_3_preds)
print(f"Mean Squared Error model_3: {mse_3}")

Mean Squared Error model_3: 8.548852881416646


In [53]:
regressorxgb = xgb.XGBRegressor(objective ='reg:squarederror')  # You can adjust hyperparameters here

scores = cross_val_score(regressorxgb, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

for fold, score in enumerate(scores):
    print(f"Fold {fold+1}: Score = {score}")

average_score = -scores.mean()
average_score

Fold 1: Score = -14.553741514753145
Fold 2: Score = -7.460015786047852
Fold 3: Score = -6.69191943123334
Fold 4: Score = -9.568706412685417
Fold 5: Score = -7.816460912704704


9.218168811484892

In [54]:
results=pd.DataFrame({
    'Model':['Decision Tree Regressor','Random Forest Regressor','XG Boost'],
    'MSE Score':[mse_1, mse_2, mse_3]
})
results_df=results.sort_values(by='MSE Score',ascending=True)
results_df=results_df.set_index('MSE Score')
results_df.head()

,Model
MSE Score,
8.548853,XG Boost
9.158102,Random Forest Regressor
17.621556,Decision Tree Regressor
